<a href="https://colab.research.google.com/github/Jamshaid9291/Tkinter-Image-Annotation-Tool-Rectangle-Point-and-Polygon-Labeling/blob/main/Tkinter_Image_Annotation_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ImageOps
import os
import json

class ImageLabelingTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Labeling Tool : Fazal Ur Rehman Azad")

        # Variables
        self.image_folder = ""
        self.image_list = []
        self.current_image_index = 0
        self.current_image = None
        self.image_tk = None
        self.annotations = {}
        self.drawing_mode = None
        self.start_x, self.start_y = None, None
        self.current_shape = None
        self.current_polygon = []
        self.undo_stack = []
        self.redo_stack = []
        self.zoom_factor = 1.0
        self.show_grid = False
        self.grid_size = 20
        self.current_color = 'red'

        # GUI Components
        self.create_widgets()

    def create_widgets(self):
        # Main frame
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Toolbar Frame
        toolbar_frame = tk.Frame(main_frame, bd=2, relief=tk.RAISED)
        toolbar_frame.grid(row=0, column=0, columnspan=2, sticky="ew")

        # Load Folder Button
        load_btn = tk.Button(toolbar_frame, text="Load Folder", command=self.load_folder)
        load_btn.pack(side=tk.LEFT, padx=2, pady=2)

        # Annotation Buttons
        rect_btn = tk.Button(toolbar_frame, text="Rectangle", command=self.set_rectangle_mode)
        rect_btn.pack(side=tk.LEFT, padx=2, pady=2)

        point_btn = tk.Button(toolbar_frame, text="Point", command=self.set_point_mode)
        point_btn.pack(side=tk.LEFT, padx=2, pady=2)

        poly_btn = tk.Button(toolbar_frame, text="Polygon", command=self.set_polygon_mode)
        poly_btn.pack(side=tk.LEFT, padx=2, pady=2)

        save_btn = tk.Button(toolbar_frame, text="Save Annotations", command=self.save_annotations)
        save_btn.pack(side=tk.LEFT, padx=2, pady=2)

        delete_btn = tk.Button(toolbar_frame, text="Delete Annotation", command=self.delete_annotation)
        delete_btn.pack(side=tk.LEFT, padx=2, pady=2)

        undo_btn = tk.Button(toolbar_frame, text="Undo", command=self.undo)
        undo_btn.pack(side=tk.LEFT, padx=2, pady=2)

        redo_btn = tk.Button(toolbar_frame, text="Redo", command=self.redo)
        redo_btn.pack(side=tk.LEFT, padx=2, pady=2)

        next_btn = tk.Button(toolbar_frame, text="Next Image", command=self.next_image)
        next_btn.pack(side=tk.LEFT, padx=2, pady=2)

        prev_btn = tk.Button(toolbar_frame, text="Previous Image", command=self.previous_image)
        prev_btn.pack(side=tk.LEFT, padx=2, pady=2)

        rotate_btn = tk.Button(toolbar_frame, text="Rotate", command=self.rotate_image)
        rotate_btn.pack(side=tk.LEFT, padx=2, pady=2)

        flip_btn = tk.Button(toolbar_frame, text="Flip", command=self.flip_image)
        flip_btn.pack(side=tk.LEFT, padx=2, pady=2)

        grid_btn = tk.Button(toolbar_frame, text="Toggle Grid", command=self.toggle_grid)
        grid_btn.pack(side=tk.LEFT, padx=2, pady=2)

        magnifier_btn = tk.Button(toolbar_frame, text="Magnifier", command=self.toggle_magnifier)
        magnifier_btn.pack(side=tk.LEFT, padx=2, pady=2)

        # Image Panel
        self.image_panel = tk.Canvas(main_frame)
        self.image_panel.grid(row=1, column=0, sticky="nsew")

        # Annotation Listbox
        self.annotation_listbox = tk.Listbox(main_frame, width=40, height=20)
        self.annotation_listbox.grid(row=1, column=1, sticky="ns")

        main_frame.grid_columnconfigure(0, weight=3)
        main_frame.grid_columnconfigure(1, weight=1)
        main_frame.grid_rowconfigure(1, weight=1)

        # Mouse Events
        self.image_panel.bind("<Button-1>", self.on_click)
        self.image_panel.bind("<B1-Motion>", self.on_drag)
        self.image_panel.bind("<ButtonRelease-1>", self.on_release)
        self.image_panel.bind("<MouseWheel>", self.on_zoom)

        # Keyboard Shortcuts
        self.root.bind("<Control-z>", lambda e: self.undo())
        self.root.bind("<Control-y>", lambda e: self.redo())
        self.root.bind("<Delete>", lambda e: self.delete_annotation())

    def load_folder(self):
        folder_selected = filedialog.askdirectory()
        if folder_selected:
            self.image_folder = folder_selected
            self.image_list = [f for f in os.listdir(folder_selected) if f.endswith(('png', 'jpg', 'jpeg'))]
            if self.image_list:
                self.current_image_index = 0
                self.load_image(self.image_list[self.current_image_index])
            else:
                messagebox.showerror("Error", "No images found in the selected folder.")

    def load_image(self, image_name):
        image_path = os.path.join(self.image_folder, image_name)
        self.current_image = Image.open(image_path)
        self.resize_and_display_image()

    def resize_and_display_image(self):
        if self.current_image is None:
            return

        window_width = self.image_panel.winfo_width()
        window_height = self.image_panel.winfo_height()

        img_width, img_height = self.current_image.size
        aspect_ratio = img_width / img_height

        if img_width > window_width or img_height > window_height:
            if img_width / window_width > img_height / window_height:
                new_width = window_width
                new_height = int(window_width / aspect_ratio)
            else:
                new_height = window_height
                new_width = int(window_height * aspect_ratio)
        else:
            new_width, new_height = img_width, img_height

        resized_image = self.current_image.resize((int(new_width * self.zoom_factor), int(new_height * self.zoom_factor)), Image.Resampling.LANCZOS)
        self.image_tk = ImageTk.PhotoImage(resized_image)
        self.image_panel.config(width=self.image_tk.width(), height=self.image_tk.height())
        self.image_panel.create_image(0, 0, anchor=tk.NW, image=self.image_tk)
        self.draw_grid()
        self.draw_annotations()

    def draw_grid(self):
        if not self.show_grid:
            return
        self.image_panel.delete("grid")
        width = self.image_panel.winfo_width()
        height = self.image_panel.winfo_height()
        for x in range(0, width, self.grid_size):
            self.image_panel.create_line(x, 0, x, height, fill="lightgrey", tags="grid")
        for y in range(0, height, self.grid_size):
            self.image_panel.create_line(0, y, width, y, fill="lightgrey", tags="grid")

    def toggle_grid(self):
        self.show_grid = not self.show_grid
        self.resize_and_display_image()

    def draw_annotations(self):
        self.image_panel.delete("annotations")
        image_name = self.image_list[self.current_image_index]
        if image_name in self.annotations:
            for annotation in self.annotations[image_name]:
                if annotation['shape'] == "rectangle":
                    self.image_panel.create_rectangle(*annotation['coords'], outline=self.current_color, tags="annotations")
                elif annotation['shape'] == "point":
                    x, y = annotation['coords']
                    self.image_panel.create_oval(x-2, y-2, x+2, y+2, outline=self.current_color, tags="annotations")
                elif annotation['shape'] == "polygon":
                    self.image_panel.create_polygon(*annotation['coords'], outline=self.current_color, fill="", tags="annotations")

    def set_rectangle_mode(self):
        self.drawing_mode = "rectangle"
        self.current_color = 'red'

    def set_point_mode(self):
        self.drawing_mode = "point"
        self.current_color = 'blue'

    def set_polygon_mode(self):
        self.drawing_mode = "polygon"
        self.current_polygon = []
        self.current_color = 'green'

    def on_click(self, event):
        if self.drawing_mode == "point":
            self.add_annotation("point", (event.x, event.y))
        elif self.drawing_mode == "polygon":
            self.current_polygon.append((event.x, event.y))
            if len(self.current_polygon) > 2:
                self.image_panel.create_polygon(*self.current_polygon, outline=self.current_color, fill="", tags="annotations")

    def on_drag(self, event):
        if self.drawing_mode == "rectangle" and self.start_x and self.start_y:
            self.image_panel.delete("temp")
            self.image_panel.create_rectangle(self.start_x, self.start_y, event.x, event.y, outline=self.current_color, tags="temp")

    def on_release(self, event):
        if self.drawing_mode == "rectangle" and self.start_x and self.start_y:
            coords = (self.start_x, self.start_y, event.x, event.y)
            self.add_annotation("rectangle", coords)
            self.image_panel.delete("temp")
        elif self.drawing_mode == "polygon":
            if len(self.current_polygon) > 2:
                self.add_annotation("polygon", self.current_polygon)

    def add_annotation(self, shape, coords):
        image_name = self.image_list[self.current_image_index]
        if image_name not in self.annotations:
            self.annotations[image_name] = []
        self.annotations[image_name].append({"shape": shape, "coords": coords})
        self.undo_stack.append((shape, coords))
        self.redo_stack.clear()
        self.update_annotation_listbox()

    def update_annotation_listbox(self):
        self.annotation_listbox.delete(0, tk.END)
        image_name = self.image_list[self.current_image_index]
        if image_name in self.annotations:
            for annotation in self.annotations[image_name]:
                self.annotation_listbox.insert(tk.END, f"{annotation['shape']}: {annotation['coords']}")

    def delete_annotation(self):
        selected_idx = self.annotation_listbox.curselection()
        if selected_idx:
            image_name = self.image_list[self.current_image_index]
            if image_name in self.annotations:
                del self.annotations[image_name][selected_idx[0]]
                self.update_annotation_listbox()

    def undo(self):
        if self.undo_stack:
            shape, coords = self.undo_stack.pop()
            self.redo_stack.append((shape, coords))
            image_name = self.image_list[self.current_image_index]
            if image_name in self.annotations:
                self.annotations[image_name].remove({"shape": shape, "coords": coords})
                self.update_annotation_listbox()
                self.draw_annotations()

    def redo(self):
        if self.redo_stack:
            shape, coords = self.redo_stack.pop()
            self.undo_stack.append((shape, coords))
            image_name = self.image_list[self.current_image_index]
            if image_name in self.annotations:
                self.annotations[image_name].append({"shape": shape, "coords": coords})
                self.update_annotation_listbox()
                self.draw_annotations()

    def on_zoom(self, event):
        if event.delta > 0:
            self.zoom_factor *= 1.1
        else:
            self.zoom_factor /= 1.1
        self.resize_and_display_image()

    def next_image(self):
        self.current_image_index = (self.current_image_index + 1) % len(self.image_list)
        self.load_image(self.image_list[self.current_image_index])

    def previous_image(self):
        self.current_image_index = (self.current_image_index - 1) % len(self.image_list)
        self.load_image(self.image_list[self.current_image_index])

    def save_annotations(self):
        output_file = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if output_file:
            with open(output_file, 'w') as f:
                json.dump(self.annotations, f)
            messagebox.showinfo("Save", "Annotations saved successfully!")

    def rotate_image(self):
        if self.current_image:
            angle = simpledialog.askinteger("Rotate", "Enter angle (degrees):")
            if angle:
                self.current_image = self.current_image.rotate(angle, expand=True)
                self.resize_and_display_image()

    def flip_image(self):
        if self.current_image:
            flip_type = simpledialog.askstring("Flip", "Enter 'h' for horizontal or 'v' for vertical:")
            if flip_type:
                if flip_type.lower() == 'h':
                    self.current_image = ImageOps.mirror(self.current_image)
                elif flip_type.lower() == 'v':
                    self.current_image = ImageOps.flip(self.current_image)
                self.resize_and_display_image()

    def toggle_magnifier(self):
        # This method is a placeholder; implement magnifier functionality here.
        pass

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageLabelingTool(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
!apt-get update
!apt-get install xvfb
!Xvfb :1 -screen 0 1024x768x24 &
import os
os.environ['DISPLAY'] = ':1'

import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ImageOps
import os
import json

class ImageLabelingTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Labeling Tool : Fazal Ur Rehman Azad")

        # Variables
        self.image_folder = ""
        self.image_list = []
        self.current_image_index = 0
        self.current_image = None
        self.image_tk = None
        self.annotations = {}
        self.drawing_mode = None
        self.start_x, self.start_y = None, None
        self.current_shape = None
        self.current_polygon = []
        self.undo_stack = []
        self.redo_stack = []
        self.zoom_factor = 1.0
        self.show_grid = False
        self.grid_size = 20
        self.current_color = 'red'

        # GUI Components
        self.create_widgets()

    def create_widgets(self):
        # Main frame
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Toolbar Frame
        toolbar_frame = tk.Frame(main_frame, bd=2, relief=tk.RAISED)
        toolbar_frame.grid(row=0, column=0, columnspan=2, sticky="ew")

        # Load Folder Button
        load_btn = tk.Button(toolbar_frame, text="Load Folder", command=self.load_folder)
        load_btn.pack(side=tk.LEFT, padx=2, pady=2)

        # Annotation Buttons
        rect_btn = tk.Button(toolbar_frame, text="Rectangle", command=self.set_rectangle_mode)
        rect_btn.pack(side=tk.LEFT, padx=2, pady=2)

        point_btn = tk.Button(toolbar_frame, text="Point", command=self.set_point_mode)
        point_btn.pack(side=tk.LEFT, padx=2, pady=2)

        poly_btn = tk.Button(toolbar_frame, text="Polygon", command=self.set_polygon_mode)
        poly_btn.pack(side=tk.LEFT, padx=2, pady=2)

        save_btn = tk.Button(toolbar_frame, text="Save Annotations", command=self.save_annotations)
        save_btn.pack(side=tk.LEFT, padx=2, pady=2)

        delete_btn = tk.Button(toolbar_frame, text="Delete Annotation", command=self.delete_annotation)
        delete_btn.pack(side=tk.LEFT, padx=2, pady=2)

        undo_btn = tk.Button(toolbar_frame, text="Undo", command=self.undo)
        undo_btn.pack(side=tk.LEFT, padx=2, pady=2)

        redo_btn = tk.Button(toolbar_frame, text="Redo", command=self.redo)
        redo_btn.pack(side=tk.LEFT, padx=2, pady=2)

        next_btn = tk.Button(toolbar_frame, text="Next Image", command=self.next_image)
        next_btn.pack(side=tk.LEFT, padx=2, pady=2)

        prev_btn = tk.Button(toolbar_frame, text="Previous Image", command=self.previous_image)
        prev_btn.pack(side=tk.LEFT, padx=2, pady=2)

        rotate_btn = tk.Button(toolbar_frame, text="Rotate", command=self.rotate_image)
        rotate_btn.pack(side=tk.LEFT, padx=2, pady=2)

        flip_btn = tk.Button(toolbar_frame, text="Flip", command=self.flip_image)
flip_btn.pack(side=tk.LEFT, padx=2, pady=2)

grid_btn = tk.Button(toolbar_frame, text="Toggle Grid", command=self.toggle_grid) # Added command to button
grid_btn.pack(side=tk.LEFT, padx=2, pady=2) # Fixed indentation

magnifier_btn = tk.Button(toolbar_frame, text="Magnifier", command=self.toggle_magnifier)
magnifier_btn.pack(side=tk.LEFT, padx=2, pady=2)

# Image Panel
self.image_panel = tk.Canvas(main_frame)
self.image_panel.grid(row=1, column=0, sticky="nsew")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac